Preparing necessary imports

In [ ]:
from ipywidgets import widgets, interact, fixed
from IPython.display import display
from getpass import getpass
import omero.gateway as gw
import omero
import pandas as pd

from plots import plot_homogeneity_map, plot_distances_map, plot_psfs, plot_distances_map_bokeh
from utils import get_microscope_history

from bokeh.io import output_notebook, reset_output, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import ColorBar, LinearColorMapper, LogColorMapper, Label

reset_output()
output_notebook()  # show output in notebook

try:
    from credentials import *
except ModuleNotFoundError:
    HOST = input('host:')
    PORT = 4064
    USER = input('user:')
    PASSWORD = getpass('password')
    GROUP = input('group:')

### Open connection

In [ ]:
conn = gw.BlitzGateway(username=USER,
                       passwd=PASSWORD,
                       host=HOST,
                       group=GROUP,
                       port=PORT)

# conn = gw.BlitzGateway(username=input('OMERO username:'), password=getpass('OMERO password:'),
#                        group=GROUP, host=OMERO_SERVER, port=PORT, secure=True)

conn.connect()
print(f'Connection successful: {conn.isConnected()}')

### Selecting the Microscope

In [ ]:
microscopes = list(conn.getObjects('Project'))
micr_menu = widgets.Dropdown(options=[(m.getName(), i) for i, m in enumerate(microscopes)],
                             value=1,
                             description='Select a microscope:',
                             disabled=False,)
display(micr_menu)

In [ ]:
microscope = microscopes[micr_menu.value]
history_df = get_microscope_history(microscope)
history_df

In [ ]:
variable_menu = widgets.Dropdown(options=list(history_df.select_dtypes(include=['int', 'float']).columns),
                                 description='Select a variable to display',
                                 disabled=False)
display(variable_menu)

In [ ]:
history_df['Acquisition_date_time'] = pd.to_datetime(history_df['Acquisition_date_time'])

p = figure(plot_width=800, plot_height=250, x_axis_type="datetime")

p.line(history_df['Acquisition_date_time'], history_df[variable_menu.value], color='navy', alpha=0.5)

show(p)